In [1]:
!pip install roboflow

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="xp4gj0yxjeXZBnmODv7M")
project = rf.workspace("cat-ksnwl").project("final_all_cat")
version = project.version(1)
dataset = version.download("yolov12")

loading Roboflow workspace...
loading Roboflow project...


In [3]:
!pip install ultralytics

import ultralytics
ultralytics.__version__

'8.3.53'

In [4]:
!pip install supervision

import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.25.1


In [5]:
import torch
torch.__version__

'2.5.1'

In [6]:
from ultralytics import YOLO
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

In [7]:
model = YOLO("yolo11l.pt")

In [8]:
data_path = 'Final_all_cat-1/data.yaml'

project = 'weights'

name = "100_ephs"

In [9]:
# Train the model
results = model.train(data=data_path,
                      project=project,
                      name=name,
                      epochs=100,
                      batch=16,
                      imgsz=640,
                      )

New https://pypi.org/project/ultralytics/8.3.85 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.53 🚀 Python-3.12.7 torch-2.5.1 CPU (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=Final_all_cat-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=weights, name=100_ephs, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sav

train: Scanning /Users/liunutt/Desktop/RA/cat/Final_all_cat-1/train/labels.cache... 6213 images, 2 backgrounds, 0 corrupt: 100%|██████████| 6213/6213 [00:00<?, ?it/s]
val: Scanning /Users/liunutt/Desktop/RA/cat/Final_all_cat-1/valid/labels.cache... 539 images, 1 backgrounds, 0 corrupt: 100%|██████████| 539/539 [00:00<?, ?it/s]


Plotting labels to weights/100_ephs/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to weights/100_ephs
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100         0G      1.516      1.863      1.879         14        640: 100%|██████████| 389/389 [5:08:20<00:00, 47.56s/it]  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [09:52<00:00, 34.86s/it]

                   all        539        573      0.186      0.351      0.141     0.0389


...

      2/100         0G      1.671      2.023      1.985         14        640: 100%|██████████| 389/389 [5:10:31<00:00, 47.90s/it]  
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [09:56<00:00, 35.12s/it]

                   all        539        573      0.327      0.407      0.272     0.0956



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      3/100         0G      1.597      1.958      1.924         42        640:  40%|███▉      | 154/389 [2:04:27<3:09:54, 48.49s/it]


KeyboardInterrupt: 

In [ ]:
models = YOLO("weights/100_ephs2/weights/best.pt")

In [ ]:
weight = models.val(
    data= '/content/drive/MyDrive/TIMS/data_train/split_5_7_24_aug/data.yaml',
    split = 'test'
)
# weight.confusion_matrix.matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

classes = [ 'cat']

plt.figure(figsize=(10, 7))
sns.heatmap(weight.confusion_matrix.matrix, annot=True, fmt=".0f", cmap="Blues", xticklabels=classes, yticklabels=classes)

plt.title('Confusion Matrix')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

In [ ]:
def compute_metrics_and_accuracy(conf_matrix):
    num_classes = conf_matrix.shape[0]
    precision = np.zeros(num_classes)
    recall = np.zeros(num_classes)
    f1_scores = np.zeros(num_classes)

    for i in range(num_classes):
        TP = conf_matrix[i, i]
        FP = conf_matrix[:, i].sum() - TP
        FN = conf_matrix[i, :].sum() - TP
        P = TP / (TP + FP) if (TP + FP) > 0 else 0
        R = TP / (TP + FN) if (TP + FN) > 0 else 0
        F1 = 2 * P * R / (P + R) if (P + R) > 0 else 0
        precision[i] = P
        recall[i] = R
        f1_scores[i] = F1

    # Compute overall accuracy
    correct_predictions = np.trace(conf_matrix)
    total_predictions = conf_matrix.sum()
    accuracy = correct_predictions / total_predictions

    return {
        "precision": precision,
        "recall": recall,
        "f1_scores": f1_scores,
        "accuracy": accuracy
    }

In [ ]:
metrix_class = compute_metrics_and_accuracy(weight.confusion_matrix.matrix[:-1,:-1])

for i, class_name in enumerate(classes):
    print(f"Class: {class_name}, Precision: {metrix_class['precision'][i]:.4f}, Recall: {metrix_class['recall'][i]:.4f}, F1 Score: {metrix_class['f1_scores'][i]:.4f}")

In [ ]:
avg_precision = np.mean(metrix_class['precision'])
avg_recall = np.mean(metrix_class['recall'])
avg_f1_score = np.mean(metrix_class['f1_scores'])

print(f"\nOverall Accuracy: {metrix_class['accuracy']:.4f}")
print(f"accuracy : {metrix_class['accuracy']}")
print(f"avg_precision : {avg_precision}")
print(f"avg_recall : {avg_recall}")
print(f"avg_f1_score : {avg_f1_score}")